In [1]:
import pandas as pd
df = pd.read_csv('hmnist_64_64_RGB.csv')

In [2]:
df.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel12279,pixel12280,pixel12281,pixel12282,pixel12283,pixel12284,pixel12285,pixel12286,pixel12287,label
0,191,152,194,191,153,195,192,149,192,194,...,185,154,180,186,156,184,182,152,173,2
1,24,13,23,24,14,28,37,24,46,61,...,26,15,29,27,15,28,24,13,25,2
2,185,129,140,192,136,151,198,142,156,198,...,152,120,138,144,113,123,115,81,84,2
3,24,11,19,36,19,30,64,38,50,91,...,27,13,16,25,12,16,24,9,14,2
4,138,94,117,158,113,138,178,133,161,191,...,140,106,118,108,77,92,67,40,55,2


In [3]:
df.shape

(10015, 12289)

In [4]:
x = df.drop("label", axis=1).values
y = df["label"].values

In [5]:
x.shape, y.shape

((10015, 12288), (10015,))

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=0)

In [7]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((8012, 12288), (2003, 12288), (8012,), (2003,))

In [8]:
x_train = x_train.reshape(x_train.shape[0], *(64, 64, 3))
x_test = x_test.reshape(x_test.shape[0], *(64, 64, 3))

In [9]:
x_train.shape, x_test.shape

((8012, 64, 64, 3), (2003, 64, 64, 3))

In [10]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [11]:
y_train.shape, y_test.shape

((8012, 7), (2003, 7))

In [12]:
#Importing keras libraries
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

#initialising model
cnn = Sequential()

#LAYER 1
#Step 1 - Convolution
cnn.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(64,64,3)))
#step 2 - Pooling
cnn.add(MaxPooling2D(pool_size=(2,2)))

#LAYER 2
#Step 1 - Convolution
cnn.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
#step 2 - Pooling
cnn.add(MaxPooling2D(pool_size=(2,2)))

#LAYER 3
#Step 1 - Convolution
cnn.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
#step 2 - Pooling
cnn.add(MaxPooling2D(pool_size=(2,2)))

#LAYER 4
#Step 1 - Convolution
cnn.add(Conv2D(filters=256, kernel_size=(3,3), activation='relu'))
#step 2 - Pooling
cnn.add(MaxPooling2D(pool_size=(2,2)))


#Flattening Layer
cnn.add(Flatten())

#Fully Connected Layer
cnn.add(Dense(units=256,activation='relu'))
cnn.add(Dense(units=128,activation='relu'))
cnn.add(Dense(units=64,activation='relu'))
cnn.add(Dense(units=32,activation='relu'))
cnn.add(Dense(units=7,activation='softmax'))

#
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 128)        0

In [13]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [16]:
history = cnn.fit(x_train,y_train,epochs=15,verbose=1, validation_split=0.2, batch_size=32)

Epoch 1/15
201/201 [==============================] - 54s 268ms/step - loss: 0.6849 - accuracy: 0.7460 - val_loss: 0.7634 - val_accuracy: 0.7293
Epoch 2/15
201/201 [==============================] - 54s 270ms/step - loss: 0.6847 - accuracy: 0.7530 - val_loss: 0.7684 - val_accuracy: 0.7224
Epoch 3/15
201/201 [==============================] - 55s 276ms/step - loss: 0.6682 - accuracy: 0.7502 - val_loss: 0.7582 - val_accuracy: 0.7205
Epoch 4/15
201/201 [==============================] - 55s 276ms/step - loss: 0.6462 - accuracy: 0.7591 - val_loss: 0.7782 - val_accuracy: 0.7230
Epoch 5/15
201/201 [==============================] - 55s 275ms/step - loss: 0.6220 - accuracy: 0.7703 - val_loss: 0.7736 - val_accuracy: 0.7149
Epoch 6/15
201/201 [==============================] - 54s 271ms/step - loss: 0.5907 - accuracy: 0.7862 - val_loss: 0.7471 - val_accuracy: 0.7367
Epoch 7/15
201/201 [==============================] - 54s 267ms/step - loss: 0.5837 - accuracy: 0.7887 - val_loss: 0.7428 - val_ac

In [17]:
import numpy as np

x_test=np.array(x_test).reshape(-1,64,64,3)
score=cnn.evaluate(x_test,y_test)
print("loss: ",score[0])
print("Accuracy: ",score[1])

63/63 [==============================] - 5s 71ms/step - loss: 1.1150 - accuracy: 0.7234
loss:  1.114950180053711
Accuracy:  0.7234148979187012


In [18]:
classes={0:('akiec', 'actinic keratoses and intraepithelial carcinomae'),
         1:('bcc' , 'basal cell carcinoma'),
         2:('bkl', 'benign keratosis-like lesions'),
         3:('df', 'dermatofibroma'),
         4:('nv', ' melanocytic nevi'),
         5:('vasc', ' pyogenic granulomas and hemorrhage'),
         6:('mel', 'melanoma'),}

In [31]:
import PIL
image=PIL.Image.open('test/mel (3).jpg')
image=image.resize((64,64))
img=x_test[1]
img=np.array(image).reshape(-1,64,64,3)
result=cnn.predict(img)
print(result[0])
result=result.tolist()
max_prob=max(result[0])
class_ind=result[0].index(max_prob)
print(classes[class_ind])

1/1 [==============================] - 0s 47ms/step
[6.3921475e-06 4.5236540e-03 2.0451401e-03 2.2779345e-04 9.4202584e-01
 2.0631733e-04 5.0964884e-02]
('nv', ' melanocytic nevi')


In [36]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(cnn)
tflite_model = converter.convert()

with open("cnn.tflite",'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\DEEPKH~1\AppData\Local\Temp\tmpa89vqa9l\assets


INFO:tensorflow:Assets written to: C:\Users\DEEPKH~1\AppData\Local\Temp\tmpa89vqa9l\assets


In [34]:
cnn.save("cnn.h5")

In [40]:
from keras.models import load_model
import tensorflow as tf

new_cnn = load_model("cnn.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(new_cnn)
tflite_model = converter.convert()

with open("cnn.tflite",'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\DEEPKH~1\AppData\Local\Temp\tmpx40coy05\assets


INFO:tensorflow:Assets written to: C:\Users\DEEPKH~1\AppData\Local\Temp\tmpx40coy05\assets


In [38]:
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)